In [1]:
import pandas as pd
import networkx as nx
import warnings
warnings.filterwarnings('ignore')

In [2]:
interact = pd.read_table("BIOGRID-CORONAVIRUS-3.5.185.tab3.txt")

In [3]:
interact.columns

Index(['#BioGRID Interaction ID', 'Entrez Gene Interactor A',
       'Entrez Gene Interactor B', 'BioGRID ID Interactor A',
       'BioGRID ID Interactor B', 'Systematic Name Interactor A',
       'Systematic Name Interactor B', 'Official Symbol Interactor A',
       'Official Symbol Interactor B', 'Synonyms Interactor A',
       'Synonyms Interactor B', 'Experimental System',
       'Experimental System Type', 'Author', 'Publication Source',
       'Organism ID Interactor A', 'Organism Name Interactor A',
       'Organism ID Interactor B', 'Organism Name Interactor B', 'Throughput',
       'Score', 'Modification', 'Qualifications', 'Tags', 'Source Database',
       'SWISS-PROT Accessions Interactor A', 'TREMBL Accessions Interactor A',
       'REFSEQ Accessions Interactor A', 'SWISS-PROT Accessions Interactor B',
       'TREMBL Accessions Interactor B', 'REFSEQ Accessions Interactor B',
       'Ontology Term IDs', 'Ontology Term Names', 'Ontology Term Categories',
       'Ontology Ter

In [4]:
interact

,#BioGRID Interaction ID,Entrez Gene Interactor A,Entrez Gene Interactor B,BioGRID ID Interactor A,BioGRID ID Interactor B,Systematic Name Interactor A,Systematic Name Interactor B,Official Symbol Interactor A,Official Symbol Interactor B,Synonyms Interactor A,...,REFSEQ Accessions Interactor A,SWISS-PROT Accessions Interactor B,TREMBL Accessions Interactor B,REFSEQ Accessions Interactor B,Ontology Term IDs,Ontology Term Names,Ontology Term Categories,Ontology Term Qualifier IDs,Ontology Term Qualifier Names,Ontology Term Types
0,798421,1387,3661,107777,109869,-,-,CREBBP,IRF3,CBP|KAT3A|RSTS,...,NP_001073315|NP_004371,Q14653,E2GIM7|E2GIM8|E2GIM5|E2GIM9|M0QYT9|E2GIM6,NP_001562|NP_001184056|NP_001184057|NP_0011840...,-,-,-,-,-,-
1,798422,3661,1387,109869,107777,-,-,IRF3,CREBBP,-,...,NP_001562|NP_001184056|NP_001184057|NP_0011840...,Q92793,-,NP_001073315|NP_004371,-,-,-,-,-,-
2,908541,59272,59272,121864,121864,UNQ868/PRO1885,UNQ868/PRO1885,ACE2,ACE2,ACEH,...,NP_068576,Q9BYF1,-,NP_068576,-,-,-,-,-,-
3,935460,3661,3661,109869,109869,-,-,IRF3,IRF3,-,...,NP_001562|NP_001184056|NP_001184057|NP_0011840...,Q14653,E2GIM7|E2GIM8|E2GIM5|E2GIM9|M0QYT9|E2GIM6,NP_001562|NP_001184056|NP_001184057|NP_0011840...,-,-,-,-,-,-
4,935461,7187,57506,113039,121570,-,-,TRAF3,MAVS,CAP-1|CAP1|CD40bp|CRAF1|IIAE5|LAP1,...,NP_663777|NP_003291|NP_001186356|NP_663778,Q7Z434,M1P2Z0,NP_065797|NP_001193420,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,2661823,5045,1489668,111082,4383915,-,SARS-CoV-2,FURIN,S,FUR|PACE|PCSK3|SPC1,...,NP_002560|NP_001276753|NP_001276752,P59594,-,NP_828851,-,-,-,-,-,-
739,2661824,5045,14254594,111082,4383879,-,G128_gp02,FURIN,S,FUR|PACE|PCSK3|SPC1,...,NP_002560|NP_001276753|NP_001276752,K9N5Q8,-,YP_009047204,-,-,-,-,-,-
740,2661825,7113,14254594,112968,4383879,-,G128_gp02,TMPRSS2,S,PP9284|PRSS10,...,NP_001128571|NP_005647,K9N5Q8,-,YP_009047204,-,-,-,-,-,-
741,2661827,43740578,43740578,4383854,4383854,GU280_gp01_nsp5,GU280_gp01_nsp5,nsp5,nsp5,ORF1ab|ORF1ab-nsp5|SARS-CoV2 nsp5|SARS-CoV-2 n...,...,YP_009725301,P0DTD1,-,YP_009725301,-,-,-,-,-,-


In [5]:
interest = ['9606', '2697049']
interact_subset = interact.query("`Organism ID Interactor A`.isin(@interest) & `Organism ID Interactor B`.isin(@interest)")

In [6]:
cols = ['Organism ID Interactor A', 'Official Symbol Interactor A']
interact_subset['Protein_A'] = interact_subset[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

cols = ['Organism ID Interactor B', 'Official Symbol Interactor B']
interact_subset['Protein_B'] = interact_subset[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

In [7]:
onlyInter = interact_subset[['Protein_A', 'Protein_B']]
onlyInter

,Protein_A,Protein_B
0,9606_CREBBP,9606_IRF3
1,9606_IRF3,9606_CREBBP
2,9606_ACE2,9606_ACE2
3,9606_IRF3,9606_IRF3
4,9606_TRAF3,9606_MAVS
...,...,...
725,9606_IKBKE,9606_TMEM173
735,9606_FURIN,2697049_S
736,9606_TMPRSS2,2697049_S
741,2697049_nsp5,2697049_nsp5


In [8]:
G=nx.from_pandas_edgelist(onlyInter, 'Protein_A', 'Protein_B')

In [9]:
degrees = dict(nx.degree(G))

In [10]:
for key, value in degrees.items():
    degrees[key] = value*1.5

In [11]:
#And finally export it to a JSON format compatible with D3
from networkx.readwrite import json_graph
import json
import re

data = json_graph.node_link_data(G)
for i in data['nodes']:
    i['species'] = int(re.sub('\_.*', '', i['id']))
    accessor = i['id']
    i['degree'] = degrees[accessor]
    
    
#And then dump it in a file
with open('biogrid.json', 'w') as fp:
    json.dump(data, fp)